# Feature Reduction Using MLXtend

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install -q mlxtend

# Load Output Activity File From PathSingle

In [ ]:
#Activity file contains the activity level for each barcode and pathway.
data = pd.read_csv('output_activity_st3.csv', index_col=0)

In [ ]:
x = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
ly = LabelEncoder()
y = ly.fit_transform(y)

# Select Different Classifiers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
#knn = KNeighborsClassifier(n_neighbors=6)
#clf = ExtraTreesClassifier(n_estimators=100)
#rfc = RandomForestClassifier(criterion="entropy")
lsvc = LinearSVC(penalty='l1', dual=False, C=2)
#gnb = GaussianNB()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=1)

# Using ForwardSelection Method

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs1 = SFS(lsvc,
           k_features=3,
           forward=True,
           floating=True,
           verbose=2,
           scoring='accuracy',
           cv=0, n_jobs=-1)

sfs1 = sfs1.fit(X_train, y_train)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 581 out of 581 | elapsed:   36.3s finished

[2023-03-19 19:27:52] Features: 1/3 -- score: 0.8003902982994145[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 580 out of 580 | elapsed:  1.5min finished

[2023-03-19 19:29:20] Features: 2/3 -- score: 0.8909952606635071[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   50.5s
[Par

In [ ]:
best_features = [int(i) for i in sfs1.k_feature_names_]
best_features

[87, 189, 503]

In [ ]:
best_features_names = data.columns[best_features]
best_features_names

Index(['ECM-receptor interaction(Kegg)',
       'Natural killer cell mediated cytotoxicity(Kegg)',
       'rb tumor suppressor/checkpoint signaling in response to dna damage(BioCarta)'],
      dtype='object')

In [ ]:
data.columns[[34, 189, 430]]

Index(['Aurora B signaling(NCI/Nature)',
       'Natural killer cell mediated cytotoxicity(Kegg)',
       'il2_stat5_test3(rotem)'],
      dtype='object')

In [ ]:
X_train_sfs = sfs1.transform(X_train)
X_test_sfs = sfs1.transform(X_test)

In [ ]:
lsvc.fit(X_train_sfs, y_train)
y_pred = lsvc.predict(X_test_sfs)

# Compute the accuracy of the prediction
acc = float((y_test == y_pred).sum()) / y_pred.shape[0]
print('Test set accuracy: %.2f %%' % (acc * 100))

Test set accuracy: 93.85 %


-------------------------------------------------------------------